<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">
 
# Ensembles and Random Forests
 
_Author: Joseph Nelson (DC)_

*Adapted from Chapter 8 of [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/)*

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline

## Getting Started

### Random Forest Regression

**Exercise (6 mins., in pairs).**

- Build a random forest regression model on the numeric features of the Ames housing dataset at `../assets/data/ames_train.csv`. You will need to import `RandomForestRegressor` from scikit-learn's `ensemble` module. Set `n_estimators=200` when you instantiate the regressor. Do a simple train/test split, train on all numeric features without null values, and get R2 values for both the training set and the test set.

- Is the model overfitting, underfitting, both, or neither? How do you know?

- **BONUS:** Create a `RandomForestRegressor` estimator that gets a better score on the test set.

$\blacksquare$

A random forest is a collection of decision trees. A random forest regressor makes a prediction by averaging the predictions from its constituent decision trees. Each tree is trained somewhat differently so that they will make different kinds of errors that will tend to cancel out.

### Random Forest Classification

**Exercise (5 mins., in pairs).**

- Build a random forest classifier model `rfc` on the numeric features of the Titanic dataset at `../assets/data/titanic.csv`. Do not use `PassengerId` as a feature. You will need to import `RandomForestClassifier` from scikit-learn's `ensemble` module. Set `n_estimators=200` and `oob_score=True` when you instantiate the classifier. Do a simple train/test split, train on all numeric features without null values, and get R2 values for both the training set and the test set.

- Is the model overfitting, underfitting, both, or neither? How do you know?

- **BONUS:** Modify the model to get a better score on the test set.

$\blacksquare$

A random forest classifier makes a prediction by taking the "majority vote" of its constituent decision trees.

## Ensembling

### Overview

**Ensemble learning (or "ensembling")** is the process of combining several predictive models in order to produce a combined model that is more accurate than any individual model. For example, given predictions from several models we could:

- **Regression:** Take the average of the predictions.
- **Classification:** Take a vote and use the most common prediction.

For ensembling to work well, the models must be:

- **Accurate:** They outperform the null model.
- **Independent:** Their predictions are generated using different processes.

**The big idea:** If you have a collection of individually imperfect (and independent) models, the "one-off" mistakes made by each model are probably not going to be made by the rest of the models, and thus the mistakes will be discarded when you average the models.

There are two basic **methods for ensembling:**

- Manually ensembling your individual models.
- Using a standard "meta-model" that does ensembling internally.

Random forests are a popular type of "meta-model."

### Manual Ensembling

What makes an effective manual ensemble?

- Different types of **models**.
- Different combinations of **features**.
- Different **tuning parameters**.

![Machine learning flowchart](../assets/images/crowdflower_ensembling.jpg)

*Machine learning flowchart created by the [winner](https://github.com/ChenglongChen/Kaggle_CrowdFlower) of Kaggle's [CrowdFlower competition](https://www.kaggle.com/c/crowdflower-search-relevance)*.

### Comparing Ensembling With a Single-Model Approach

**Advantage of ensembling:** it can increase predictive accuracy.

**Disadvantages of ensembling:**

- It decreases interpretability.
- It takes longer to train.
- It takes longer to predict.
- It is more complex to automate and maintain, particularly with manual ensembling.

Outside of machine learning competitions, you have to weigh gains in accuracy against added complexity.

### Bagging

The primary weakness of **decision trees** is that they don't tend to have the best predictive accuracy. This is partially because of **high variance**, meaning that different splits in the training data can lead to very different trees.

**Bagging** is a general-purpose procedure for reducing the variance of a machine learning method but is particularly useful for decision trees. Bagging is short for **bootstrap aggregation**, meaning the aggregation of bootstrap samples.

A **bootstrap sample** is a random sample with replacement. So, it has the same size as the original sample but might duplicate some of the original observations.

In [ ]:
# Set a seed for reproducibility.
np.random.seed(1)

In [ ]:
# Create an array of 1 through 20.


In [ ]:
# Sample that array 20 times with replacement.


**How does bagging work (for decision trees)?**

1. Grow B trees using B bootstrap samples from the training data.
2. Train each tree on its bootstrap sample and make predictions.
3. Combine the predictions:
    - Average the predictions for **regression trees**.
    - Take a vote for **classification trees**.

**Notes:**

- **Each bootstrap sample** is typically the same size as the original training set. (It may contain repeated rows.)
- **B** should be a large enough value that the error seems to have "stabilized".
- The trees are **grown deep** so that they have low bias/high variance.

Training multiple trees through bagging can give more consistent results than training a single tree, thereby reducing variance.

#### Manually Implementing Bagged Decision Trees (with B=10)

In [ ]:
# Read in and prepare the vehicle training data.
path = '../assets/data/vehicles_train.csv'
train = pd.read_csv(path)

In [ ]:
# Transform "vtype" to "is_truck"


In [ ]:
# Set a seed for reproducibility.
np.random.seed(123)

In [ ]:
# Create ten bootstrap samples (which will be used to select rows from the DataFrame).


In [ ]:
# Show the rows for the first decision tree.


In [ ]:
# Read in and prepare the vehicle testing data.


In [ ]:
# Define testing data.


In [ ]:
# Import decision tree regressor


In [ ]:
# Grow one tree for each bootstrap sample and make predictions on testing data.


In [ ]:
# Convert predictions from list to NumPy array.


In [ ]:
# Average predictions.


In [ ]:
# Calculate RMSE.


#### Bagged Decision Trees in `scikit-learn` (with B=500)

In [ ]:
# Define the training and testing sets.


In [ ]:
# Instruct BaggingRegressor to use DecisionTreeRegressor as the "base estimator."


In [ ]:
# Fit and predict.


In [ ]:
# Calculate RMSE.


### Random Forests

Random Forests offer a **slight variation on bagged trees** that usually gives better performance:

- Exactly like bagging, we create an ensemble of decision trees using bootstrapped samples of the training set.
- However, when building each tree, each time a split is considered, a **random sample of m features** is chosen as split candidates from the **full set of p features**. The split is only allowed to use **one of those m features**.
    - A new random sample of features is chosen for **every single tree at every single split**.
    - For **classification**, m is typically chosen to be the square root of p.
    - For **regression**, m is typically chosen to be somewhere between p/3 and p.

What's the point?

- Suppose there is **one very strong feature** in the data set. When using bagged trees, most of the trees will use that feature as the top split, resulting in an ensemble of similar trees that are **highly correlated**.
- Averaging highly correlated quantities does not significantly reduce variance (which is the entire goal of bagging).
- By randomly leaving out candidate features from each split, **random forests "decorrelate" the trees** to the extent that the averaging process can reduce the variance of the resulting model.
- Another way of looking at it is that sometimes one or two strong features dominate every tree in bagging, resulting in essentially the same tree as every predictor. (This is what was meant when saying the trees could be highly correlated.) By using a subset of features to generate each tree, we get a wider variety of predictive trees that do not all use the same dominant features.

### Out-of-Bag Error

The primary purpose of ensembling is to create a "team" of models that performs better than its individual members.

A **side benefit** of bagging is that it allows you to estimate out-of-sample errorwithout using a train/test split or cross-validation!

For each tree, the observations that were not used in training are called **"out-of-bag" observations.**

In [ ]:
# Show the first bootstrap sample.


In [ ]:
# Show the "in-bag" observations for each sample.


In [ ]:
# Show the "out-of-bag" observations for each sample.


**Calculating "out-of-bag error:"**

1. For each observation in the training data, predict its response value using **only** the trees in which that observation was out-of-bag. Average those predictions (for regression) or take a vote (for classification).
2. Compare all predictions to the actual response values in order to compute the out-of-bag error.

When B is sufficiently large, the **out-of-bag error** is an accurate estimate of **out-of-sample error**.

In [ ]:
# Compute the out-of-bag R-squared score for `rfc`.


### Feature Importance

Bagging increases **predictive accuracy** but decreases **model interpretability** because it's no longer possible to visualize the tree to understand the importance of each feature.

However, we can still obtain an overall summary of **feature importance** from bagged models:

- **Bagged regression trees:** Calculate the total amount that **MSE** decreases due to splits over a given feature, averaged over all trees
- **Bagged classification trees:** Calculate the total amount that **Gini index** decreases due to splits over a given feature, averaged over all trees

In [ ]:
list(zip(titanic.select_dtypes(['int64', 'float64'])
         .dropna(axis='columns')
         .drop(['Survived', 'PassengerId'], axis='columns'),
    rfc.feature_importances_)
)

**Exercise (6 mins.)**

In your own words...

- What is ensembling?

- How do bagged classification trees work?

- How does a random forest differ from an ordinary bagged decision tree?

- What is out-of-bag error?

- Compare and contrast ensembling and K-fold cross validation in terms of both process and aims.

$\blacksquare$

## Building and Tuning Random Forests

### Preparing the Data

In [ ]:
# Read in the hitters data.
path = '../assets/data/hitters.csv'
hitters = pd.read_csv(path)

In [ ]:
# Remove rows with missing values.


In [ ]:
# Encode categorical variables as integers.


In [ ]:
# Create a scatter plot of hits vs years, colored by salary


In [ ]:
# Define features: Exclude career statistics (which start with "C") and the response (salary).


In [ ]:
# Define X and y.


### Predicting Salary With a Decision Tree

Let's first recall how we might predict salary using a single decision tree.

We'll first find the best **max_depth** for a decision tree using cross-validation:

In [ ]:
# List of values to try for max_depth:


In [ ]:
# Use 10-fold cross-validation with each value of max_depth.


In [ ]:
# Plot max_depth (x-axis) versus R^2 (y-axis).


In [ ]:
# Show the best RMSE and the corresponding max_depth.


In [ ]:
# max_depth=2 was best, so fit a tree using that parameter.


In [ ]:
# Compute feature importances.


### Predicting Salary With a Random Forest

In [ ]:
# Import random forest regressor


In [ ]:
# max_features=5 is best and n_estimators=150 is sufficiently large.


In [ ]:
# Compute feature importances.


In [ ]:
# Compute the out-of-bag R-squared score.


**Exercise (12 mins., in pairs)**

- Use a `RandomForestRegressor` estimator to build a quick first model to predict "count" in the `bikes` dataset. Be sure not to use "casual" and "registered" as features. Measure R^2 using a random test set, ignoring the time-series aspects of the dataset.

In [ ]:
path = '../assets/data/bikeshare.csv'
bikes = pd.read_csv(path)

- **BONUS:** Make your model more realistic by taking the test set from the end and time-shifting the variables that cannot be known in advance by two hours.

- **BONUS:** Improve your model's score, e.g. through feature engineering.

*Tip:* If you have a small gap between training-set performance and test-set performance, then focus on decreasing bias. Otherwise, focus on decreasing variance.

$\blacksquare$

### Hyperparameter Tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfreg = RandomForestRegressor()
rfreg

#### Tuning n_estimators

One important tuning parameter is **n_estimators**, which represents the number of trees that should be grown. This should be a large enough value that the error seems to have "stabilized."

In [ ]:
# List of values to try for n_estimators:
estimator_range = list(range(10, 310, 10))

# List to store the average RMSE for each value of n_estimators:
RMSE_scores = []

# Use five-fold cross-validation with each value of n_estimators (Warning: Slow!).
for estimator in estimator_range:
    rfreg = RandomForestRegressor(n_estimators=estimator, random_state=1)
    MSE_scores = cross_val_score(rfreg, X, y, cv=5, scoring='neg_mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

In [ ]:
# Plot RMSE (y-axis) versus n_estimators (x-axis).

fig, ax = plt.subplots()
ax.plot(estimator_range, RMSE_scores);
ax.set_xlabel('n_estimators');
ax.set_ylabel('RMSE (lower is better)');

**Adding more trees will only help average performance, with diminishing returns.**

#### Tuning max_features

The other important tuning parameter is **max_features**, which represents the number of features that should be considered at each split.

In [ ]:
# List of values to try for max_features:
feature_range = list(range(1, len(feature_cols)+1))

# List to store the average RMSE for each value of max_features:
RMSE_scores = []

# Use 10-fold cross-validation with each value of max_features (Warning: Super slow!).
for feature in feature_range:
    rfreg = RandomForestRegressor(n_estimators=150, max_features=feature, random_state=1)
    MSE_scores = cross_val_score(rfreg, X, y, cv=10, scoring='neg_mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

In [ ]:
# Plot max_features (x-axis) versus RMSE (y-axis).
fig, ax = plt.subplots()
ax.plot(feature_range, RMSE_scores);
ax.set_xlabel('max_features');
ax.set_ylabel('RMSE (lower is better)');

In [ ]:
# Show the best RMSE and the corresponding max_features.
sorted(zip(RMSE_scores, feature_range))[0]

## Comparing Random Forests to Decision Trees

**Advantages of random forests:**

- Their performance is often competitive with the best supervised learning methods, unlike that of decision trees.
- They provide a more reliable estimate of feature importance.
- They allow you to estimate out-of-sample error without using train/test split or cross-validation.

**Disadvantages of random forests:**

- They are less interpretable.
- They are slower to train.
- They are slower to predict.

**Which model is best?** The best classifier for a particular task is task-dependent. In many business cases, interpretability is more important than accuracy. So, decision trees may be preferred. In other cases, accuracy on unseen data might be paramount, in which case random forests would likely be better (since they typically overfit less). 

## Summary

- Ensembling combines several predictive models with the goal of improving accuracy.
- For ensembling to work, the component models need to have different patterns of error that tend to cancel each other out.
- Bagging produces an ensemble of models (often decision trees) by training each one on a bootstrap sample from the original dataset.
- A side benefit of bagging is that out-of-bag scores can be used to estimate generalization performance without a train/test split or cross-validation.
- A random forest is an ensemble of decision trees trained using both bagging and feature subsetting.